In [38]:
import os
import pandas as pd
import openpyxl, uuid, datetime, getpass, json

# Assumptions
#
# if unable to tell, use the first sheet


In [39]:
# Read files in the input directory for Excel items without accompanying .json document

wlk = {};
for r,d,f in os.walk('input'):
    for file in f:
        if file.endswith(".xlsx") and not file.startswith('~'):
            wlk[file] = True;
      
    for file in f:
        if file.endswith(".json") and os.path.splitext(file)[0] + '.xlsx' in wlk:
            wlk[os.path.splitext(file)[0] + '.xlsx'] = False;
 
rez = {};
for k,v in wlk.items():
    if v is True:
        rez[k] = {};
        
print("Found " + str(len(rez)) + " Excel files to process.")
for k in list(rez.keys()):
    print("  " + k);
    

Found 13 Excel files to process.
  CA_LUST_template.xlsx
  CA_UST_template.xlsx
  NC_LUST_template.xlsx
  NC_UST_template.xlsx
  NE_UST_template.xlsx
  OR_LUST.xlsx
  OR_UST.xlsx
  SC_LUST_template.xlsx
  SC_UST_template.xlsx
  TN_LUST_template.xlsx
  TN_UST_template.xlsx
  TrUSTD_LUST_template.xlsx
  TrUSTD_UST_template.xlsx


In [40]:
# Determine whether UST or LUST

for k,v in rez.items():
    print("Examining " + k);
    rez[k]['key'] = k;
    rez[k]['file'] = 'input' + os.sep + k;
    rez[k]['date_examined'] = datetime.datetime.today().strftime('%Y%m%d');
    rez[k]['username'] = getpass.getuser();
    certainty = 0;
    
    lowname = k.lower();

    if lowname.find('_lust.') > 0 or lowname.find('_lust_') > 0:
        rez[k]['datatype'] = 'LUST';
        certainty = 100;
        
    elif lowname.find('_ust.') > 0 or lowname.find('_ust_') > 0:
        rez[k]['datatype'] = 'UST';
        certainty = 100;
        
    elif lowname.find('_lust') > 0:
        rez[k]['datatype'] = 'LUST';
        certainty = 80;
        
    elif lowname.find('_ust') > 0:
        rez[k]['datatype'] = 'UST';
        certainty = 80;
        
    elif lowname.find('lust') > 0:
        rez[k]['datatype'] = 'LUST';
        certainty = 40;
        
    else:
        print("  unable to tell datatype from filename.");
        certainty = 0;
        
    if certainty < 100:
        xlsx = pd.ExcelFile('input' + os.sep + k);
        
        print("  checking sheet names for clues");
        for sht in xlsx.sheet_names:
            lowname = sht.lower();
            
            if lowname.find('_lust') > 0:
                rez[k]['datatype'] = 'LUST';
                certainty = 100;
                break;
            
            elif lowname.find(' lust') > 0:
                rez[k]['datatype'] = 'LUST';
                certainty = 100;
                break;
                
            elif lowname.find('_ust') > 0:
                rez[k]['datatype'] = 'UST';
                certainty = 100;
                break;
                
            elif lowname.find(' ust') > 0:
                rez[k]['datatype'] = 'UST';
                certainty = 100;
                break;
                
            if 'datatype' in rez[k]:
                if rez[k]['datatype'] == 'LUST' and lowname.find('lust') > 0:
                    rez[k]['datatype'] = 'LUST';
                    certainty = 100;
                    break;

                elif rez[k]['datatype'] == 'UST' and lowname.find('ust') > 0:
                    rez[k]['datatype'] = 'UST';
                    certainty = 100;
                    break;
                    
        del xlsx
    
    if certainty == 0:    
        rez[k]['error'] = 'Unable to determine UST or LUST';
        
    if 'error' in rez[k]:
        print("  " + rez[k]['error']);
    else:
        print("  determined datatype as " + rez[k]['datatype'] + ".");


Examining CA_LUST_template.xlsx
  determined datatype as LUST.
Examining CA_UST_template.xlsx
  determined datatype as UST.
Examining NC_LUST_template.xlsx
  determined datatype as LUST.
Examining NC_UST_template.xlsx
  determined datatype as UST.
Examining NE_UST_template.xlsx
  determined datatype as UST.
Examining OR_LUST.xlsx
  determined datatype as LUST.
Examining OR_UST.xlsx
  determined datatype as UST.
Examining SC_LUST_template.xlsx
  determined datatype as LUST.
Examining SC_UST_template.xlsx
  determined datatype as UST.
Examining TN_LUST_template.xlsx
  determined datatype as LUST.
Examining TN_UST_template.xlsx
  determined datatype as UST.
Examining TrUSTD_LUST_template.xlsx
  determined datatype as LUST.
Examining TrUSTD_UST_template.xlsx
  determined datatype as UST.


In [41]:
# Determine whether STATE or TRIBE

states = [
     'AK','AZ','AR','AS','CA','CO','CT','DE','DC','FL','GA','GU','HI','ID','IL','IN','IA'
    ,'KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MP','MT','NE','NV','NH','NJ','NM'
    ,'NY','NC','ND','OH','OK','OR','PA','PR','RI','SC','SD','TN','TX','UT','VT','VI','VA'
    ,'WA','WV','WI','WY'
]

for k,v in rez.items():
    print("Examining " + k);
    rez[k]['key'] = k;
    
    lowname = k.lower();
    upname  = k.upper();
    
    if lowname[0:2] == 'tr':
        rez[k]['organization'] = 'TRIBE';
        rez[k]['state'] = None;
        
    else:
        rez[k]['organization'] = 'STATE';
        
        if upname[0:2] in states:
            rez[k]['state'] = upname[0:2];
            
        else:
            rez[k]['error'] = 'Unable to discern state or tribal location.';
        
    if 'error' in rez[k]:
        print("  " + rez[k]['error']);
    else:
        print("  determined organization as " + rez[k]['organization'] + " " + str(rez[k]['state']) + ".");
    

Examining CA_LUST_template.xlsx
  determined organization as STATE CA.
Examining CA_UST_template.xlsx
  determined organization as STATE CA.
Examining NC_LUST_template.xlsx
  determined organization as STATE NC.
Examining NC_UST_template.xlsx
  determined organization as STATE NC.
Examining NE_UST_template.xlsx
  determined organization as STATE NE.
Examining OR_LUST.xlsx
  determined organization as STATE OR.
Examining OR_UST.xlsx
  determined organization as STATE OR.
Examining SC_LUST_template.xlsx
  determined organization as STATE SC.
Examining SC_UST_template.xlsx
  determined organization as STATE SC.
Examining TN_LUST_template.xlsx
  determined organization as STATE TN.
Examining TN_UST_template.xlsx
  determined organization as STATE TN.
Examining TrUSTD_LUST_template.xlsx
  determined organization as TRIBE None.
Examining TrUSTD_UST_template.xlsx
  determined organization as TRIBE None.


In [42]:
# Determine the sheet to process

for k,v in rez.items():
        
    print("Examining " + k);
    xlsx = pd.ExcelFile('input' + os.sep + k);
    sht_count = len(xlsx.sheet_names)
    print("File has " + str(sht_count) + " sheets");

    if sht_count == 1:
        rez[k]['sheet_no']    = 0;
        rez[k]['sheet_name'] = xlsx.sheet_names[0];

    else:
        idx = 0;
        for sht in xlsx.sheet_names:
            lowname = sht.lower();
 
            if rez[k]['datatype'] == 'LUST' and lowname.find(' lust') > 0:
                rez[k]['sheet_no']   = idx;
                rez[k]['sheet_name'] = sht;
                break;

            elif rez[k]['datatype'] == 'LUST' and lowname.find('_lust') > 0:
                rez[k]['sheet_no']   = idx;
                rez[k]['sheet_name'] = sht;
                break;

            elif rez[k]['datatype'] == 'UST' and lowname.find('_ust') > 0:
                rez[k]['sheet_no']   = idx;
                rez[k]['sheet_name'] = sht;
                break;

            elif rez[k]['datatype'] == 'UST' and lowname.find(' ust') > 0:
                rez[k]['sheet_no']   = idx;
                rez[k]['sheet_name'] = sht;
                break;
                
            idx += 1;

        if rez[k]['sheet_no'] is None:
            rez[k]['error'] = 'Unable to determine sheet to process';

    del xlsx  
        
    if 'error' in rez[k]:
        print("  " + rez[k]['error']);
    else:
        print("  will process " + rez[k]['sheet_name'] + ", index " + str(rez[k]['sheet_no']));
          

Examining CA_LUST_template.xlsx
File has 10 sheets
  will process CA_LUST, index 1
Examining CA_UST_template.xlsx
File has 11 sheets
  will process CA_UST, index 1
Examining NC_LUST_template.xlsx
File has 8 sheets
  will process NC LUST, index 1
Examining NC_UST_template.xlsx
File has 7 sheets
  will process NC_UST, index 1
Examining NE_UST_template.xlsx
File has 9 sheets
  will process NE UST, index 6
Examining OR_LUST.xlsx
File has 7 sheets
  will process OR_LUST, index 0
Examining OR_UST.xlsx
File has 12 sheets
  will process OR UST, index 0
Examining SC_LUST_template.xlsx
File has 8 sheets
  will process SC LUST, index 0
Examining SC_UST_template.xlsx
File has 6 sheets
  will process SC UST, index 0
Examining TN_LUST_template.xlsx
File has 10 sheets
  will process TN LUST, index 0
Examining TN_UST_template.xlsx
File has 6 sheets
  will process TN_UST, index 0
Examining TrUSTD_LUST_template.xlsx
File has 9 sheets
  will process TrUST LUST, index 0
Examining TrUSTD_UST_template.xlsx


In [43]:
# Determine fields to geocode

for k,v in rez.items():
    if 'error' not in v:
        
        print("Examining " + rez[k]['datatype'] + ' file ' + k + ', sheet ' + str(rez[k]['sheet_no']));
        df = pd.read_excel('input' + os.sep + k,sheet_name=rez[k]['sheet_no']);
        tr = len(df.index);
        print("Sheet has " + str(tr) + " records.");
        
        rez[k]['record_count'] = tr;
        rez[k]['mapping'] = {};
        rez[k]['has_globalid'] = False;
        mp = rez[k]['mapping'];
        
        for col in df.columns:
            lowname = col.lower();
            
            if lowname == 'globalid':
                rez[k]['has_globalid'] = True;
            
            # make sure we only take the first matching instance from left to right
            elif rez[k]['datatype'] == 'UST':
                if 'sourceidentifier' not in mp and lowname == 'facilityid':
                    mp['sourceidentifier'] = col;
                    
                elif 'sourceidentifier2' not in mp and lowname == 'tankid':
                    mp['sourceidentifier2'] = col;
             
                elif 'name' not in mp and lowname == 'facilityname':
                    mp['name'] = col;
                
                elif 'address1' not in mp and lowname in ['facilityaddress','facilityaddress1']:
                    mp['address1'] = col;
                    
                elif 'address2' not in mp and lowname == 'facilityaddress2':
                    mp['address2'] = col;
            
                elif 'address3' not in mp and lowname == 'facilityaddress3':
                    mp['address3'] = col;
                    
                elif 'city' not in mp and lowname == 'facilitycity':
                    mp['city'] = col;
                    
                elif 'county' not in mp and lowname == 'facilitycounty':
                    mp['county'] = col;
                    
                elif 'state' not in mp and lowname == 'facilitystate':
                    mp['state'] = col;
                    
                elif 'zip5' not in mp and lowname in ['facilityzipcode','zipcode']:
                    mp['zip5'] = col;
                    
                elif 'zip4' not in mp and lowname == 'zip4':
                    mp['zip4'] = col;
                    
                elif 'phone' not in mp and lowname in ['facilityphone','facilityphonenumber']:
                    mp['phone'] = col;
                    
                elif 'tribe' not in mp and lowname == 'facilitytribename':
                    mp['tribe'] = col;
                    
                elif 'originallat' not in mp and lowname == 'facilitylatitude':
                    mp['originallat'] = col;
                    
                elif 'originallng' not in mp and lowname == 'facilitylongitude':
                    mp['originallng'] = col;
                    
                elif 'eparegion' not in mp and lowname == 'facilityeparegion':
                    mp['eparegion'] = col;
                    
            elif rez[k]['datatype'] == 'LUST':
                if 'sourceidentifier' not in mp and lowname == 'facilityid':
                    mp['sourceidentifier'] = col;
                    
                elif 'sourceidentifier2' not in mp and lowname == 'lustid':
                    mp['sourceidentifier2'] = col;
             
                elif 'name' not in mp and lowname == 'sitename':
                    mp['name'] = col;
                
                elif 'address1' not in mp and lowname in ['siteaddress','siteaddress1']:
                    mp['address1'] = col;
                    
                elif 'address2' not in mp and lowname == 'siteaddress2':
                    mp['address2'] = col;
            
                elif 'address3' not in mp and lowname == 'siteaddress3':
                    mp['address3'] = col;
                    
                elif 'city' not in mp and lowname == 'sitecity':
                    mp['city'] = col;
                    
                elif 'county' not in mp and lowname in ['sitecounty','county']:
                    mp['county'] = col;
                    
                elif 'state' not in mp and lowname in ['sitestate','state']:
                    mp['state'] = col;
                    
                elif 'zip5' not in mp and lowname in ['sitezipcode','zipcode']:
                    mp['zip5'] = col;
                
                elif 'zip4' not in mp and lowname == 'zip4':
                    mp['zip4'] = col;
                    
                elif 'phone' not in mp and lowname == 'sitephone':
                    mp['phone'] = col;
                    
                elif 'tribe' not in mp and lowname == 'tribename':
                    mp['tribe'] = col;
                    
                elif 'originallat' not in mp and lowname == 'latitude':
                    mp['originallat'] = col;
                    
                elif 'originallng' not in mp and lowname == 'longitude':
                    mp['originallng'] = col;
                    
                elif 'eparegion' not in mp and lowname == 'eparegion':
                    mp['eparegion'] = col;
       
        del df;
        for k1,v1 in sorted(mp.items()):
            print("  " + k1 + ": " + v1);
        print(" ");
        

Examining LUST file CA_LUST_template.xlsx, sheet 1
Sheet has 42049 records.
  address1: SiteAddress
  city: SiteCity
  county: County
  eparegion: EPARegion
  name: SiteName
  originallat: Latitude
  originallng: Longitude
  sourceidentifier: FacilityID
  sourceidentifier2: LUSTID
  state: State
  zip5: Zipcode
 
Examining UST file CA_UST_template.xlsx, sheet 1
Sheet has 32923 records.
  address1: FacilityAddress1
  city: FacilityCity
  county: FacilityCounty
  eparegion: FacilityEPARegion
  name: FacilityName
  originallat: FacilityLatitude
  originallng: FacilityLongitude
  sourceidentifier: FacilityID
  sourceidentifier2: TankID
  state: FacilityState
  zip5: FacilityZipCode
 
Examining LUST file NC_LUST_template.xlsx, sheet 1
Sheet has 17555 records.
  address1: SiteAddress
  city: SiteCity
  county: County
  eparegion: EPARegion
  name: SiteName
  originallat: Latitude
  originallng: Longitude
  sourceidentifier: FacilityID
  sourceidentifier2: LUSTID
  state: State
  zip5: Zipcod

In [44]:
# Add globalids if not present
for k,v in rez.items():
    if 'error' not in v:
        
        if not v['has_globalid']:
            print("File " + k + ", sheet " + str(v['sheet_no']) + " lacks globalids.");
            
            wb = openpyxl.load_workbook('input' + os.sep + k);
            sheets = wb.sheetnames;

            ws = wb[sheets[v['sheet_no']]];
            
            print("Adding globalids (may take a while)...");
            newcol = ws.max_column + 1;
            ws.cell(
                 row    = 1
                ,column = newcol
                ,value  = 'globalid'
            );
            for row_num in range(2,ws.max_row+1):
                ws.cell(
                     row    = row_num
                    ,column = newcol
                    ,value  = '{' + str(uuid.uuid4()) + '}'
                );
            
            print("Saving workbook.");
            wb.save('input' + os.sep + k);
            del ws,wb;
            v['has_globalid'] = True;
            
        else:
            print("File " + k + ", sheet " + str(v['sheet_no']) + " has preexisting globalids.");
            

File CA_LUST_template.xlsx, sheet 1 has preexisting globalids.
File CA_UST_template.xlsx, sheet 1 has preexisting globalids.
File NC_LUST_template.xlsx, sheet 1 has preexisting globalids.
File NC_UST_template.xlsx, sheet 1 has preexisting globalids.
File NE_UST_template.xlsx, sheet 6 has preexisting globalids.
File OR_LUST.xlsx, sheet 0 has preexisting globalids.
File OR_UST.xlsx, sheet 0 has preexisting globalids.
File SC_LUST_template.xlsx, sheet 0 has preexisting globalids.
File SC_UST_template.xlsx, sheet 0 has preexisting globalids.
File TN_LUST_template.xlsx, sheet 0 has preexisting globalids.
File TN_UST_template.xlsx, sheet 0 has preexisting globalids.
File TrUSTD_LUST_template.xlsx, sheet 0 has preexisting globalids.
File TrUSTD_UST_template.xlsx, sheet 0 has preexisting globalids.


In [45]:
# Write out the companion json file

for k,v in rez.items():
    if 'error' not in v:
        
        fn = 'input' + os.sep + os.path.splitext(k)[0] + '.json';
        if os.path.exists(fn):
            os.remove(fn);
        
        print("writing out companion file for " + k);
        with open(fn, 'w') as fp:
            json.dump(v,fp,indent = 3);
        

writing out companion file for CA_LUST_template.xlsx
writing out companion file for CA_UST_template.xlsx
writing out companion file for NC_LUST_template.xlsx
writing out companion file for NC_UST_template.xlsx
writing out companion file for NE_UST_template.xlsx
writing out companion file for OR_LUST.xlsx
writing out companion file for OR_UST.xlsx
writing out companion file for SC_LUST_template.xlsx
writing out companion file for SC_UST_template.xlsx
writing out companion file for TN_LUST_template.xlsx
writing out companion file for TN_UST_template.xlsx
writing out companion file for TrUSTD_LUST_template.xlsx
writing out companion file for TrUSTD_UST_template.xlsx
